# *Measuring Three-Dimensional Carpal Alignment from Weight-Bearing Computed Tomography Images*

- By: *Justen Saini, Brodie Ritchie, Neil J. White, Koren E. Roach, Sarah L. Manske; University of Calgary.* For more information on our lab's work, please visit: https://www.manskelab.ca/
- *Included as part of the manuscript 'Three-Dimensional Morphology-Based Approaches for Measuring Dorsal Scaphoid Translation'*. This work was submitted to the Journal of Orthopaedic Research (JOR) in February 2026.*
- Code license: *GNU GPL v3.0*  

- Narrative license: *CC-BY 4.0*  

- How to cite: [![DOI](https://zenodo.org/badge/1160168873.svg)](https://doi.org/10.5281/zenodo.18672968)

---
# Aims

- 1) __Loading Data__: Read in meshes from CT scans -- either in .nii or .vtk format
- 2) __Defining Bone Properties__: Define morphological and inertial properties of the carpal bone meshes
        - This includes principal curvature, the centroid, and the principal axes of inertia
- 3) __Defining Axes__: Define the mesh- and axis-related functions required to measure carpal alignment axes
- 4) Output carpal alignment measurements
        - Optionally, output the measurements to a CSV file

---
# Table of Contents

  1. [Installations and Dependencies](#Installations-and-Dependencies)
  2. [Global Variables and Function Definitions](#Functions-and-Global-Variables-Definitions*)  
  3. [Mesh Development and Processing](#Mesh-Development-and-Processing)  
  4. [Carpal Alignment Definitions](#Carpal-Alignment-Definitions)  
  5. [Results](#Results)  
  6. [References](#References)  
  7. [Acknowledgments](#Acknowledgements)  



---
<a name="Installations"></a>
## *Installations and Dependencies*

This section details the required and optional installations for the libraries used in this work.

I can also add information on how to install the Manskelab environment or how to install the various libraries on PC/Mac here.

##### **Required Installations**

In [ ]:
# Installations required for the script to run:

import os
import sys
import vtk
import SimpleITK as sitk 
from vtk.util.numpy_support import vtk_to_numpy
import pyvista as pv
import numpy as np
import time
from sklearn.decomposition import PCA
from pyacvd import Clustering
import trame
import pandas as pd


##### **Optional Installations**

These are additional imports that may be useful in future implementations of this script, but are not sepcifically required for this project.

In [ ]:
# Optional library installations:

import argparse 
import random
from scipy import ndimage as ndi
import open3d as o3d 
from vtk.util.numpy_support import numpy_to_vtk
from scipy.spatial import distance, KDTree, distance_matrix
from matplotlib import pyplot as plt
from collections import defaultdict

##### **Python Function Installations**

---
<a name="Functions-and-Global-Variables"></a>
## *Functions and Global Variables Definitions*

This section includes the functions used during various components of the carpal alignment calculations below, as well as global variables that are used at multiple times throughout the script.

##### **Carpal Alignment Function Definitions**

In [ ]:
# Reading in the Python scripts that contain functions used in this main script

from calc import calculate_2D_angle, calculate_3D_angle, nearest_neighbour_distance
from curvature import calculate_curvatures, normalize_curvatures
from mesh_division import half_mesh, divide_mesh_into_sections
from mesh_processing import principal_component_analysis, compute_derivatives, calculate_vertex_normals, compute_mass_moment_of_inertia, extract_mesh_from_vtk, smooth_mesh, decimate_mesh
from point_finding import compute_bone_centroid
from bone_transformations import crop_meshes, landmark_transform, icp_transform, transform_uncropped_mesh
from read_vtk import read_vtk

from axis_definitions import get_radial_axes, get_scaphoid_axis, get_lunate_axis, get_capitate_axis, get_mc3_axis
from axis_definitions import adjust_scaphoid_principal_axes, adjust_lunate_principal_axes, adjust_capitate_principal_axes, adjust_mc3_principal_axes
from surface_definitions import define_articulating_surface
from dst_definitions import get_3D_dst_components, measure_dtl_dst, measure_prsa_dst




##### **Global Variables**

In [ ]:
# Defining various parameters used throughout the code


SLIL_INJURY = False # Indicate if the participant you are evaluating has an SLIL injury - this will impact the angle measurements used for the scapholunate interval

# Variables used for mesh processing and point finding
CURVATURE_RADIUS = 5
LUNATE_REFINEMENT_SEARCH_RADIUS = 10
SCAPHOID_LENGTH_MULTIPLIER = 1.5 # This is used to extend the scaphoid axis for visualization purposes
LUNATE_LENGTH_MULTIPLIER = 1.5 # This is used to extend the lunate axis for visualization purposes
MC3_LENGTH_MULTIPLIER = 2 # This is used to extend the MC3 axis for visualization purposes
COM_THRESHOLD = 0.5 # A threshold (in mm) used for each slice of the bone when calculating the center of mass along an axis

SCAPHOID_DIVISIONS = 3
LUNATE_DIVISIONS = 3

# Adjust these parameters to align the radius with the coordinates of the global coordinate system -- these are used as initial estimates
RADIUS_SHAFT_DIRECTION = "Y" # E.g. This should be set to "Y" for datasets where the shaft of the radius is aligned with the global Y-axis
RADIAL_ULNAR_DIRECTION = "X" 
PALMAR_DORSAL_DIRECTION = "Z"


SL_ARTICULATION_THRESHOLD = 5.0 # The initial distance threshold (in mm) used to define the scapholunate articulation
if SLIL_INJURY == True: # The angle thresholds used to define the scapholunate articulation will differ if there is an SLIL injury
    MIN_SL_ANGLE = 105
    MAX_SL_ANGLE = 255
else:
    MIN_SL_ANGLE = 120
    MAX_SL_ANGLE = 240
SCENE_DISTANCE = 200




---
<a name="Meshes"></a>
## *Mesh Development and Processing*

##### **Load the carpal bone meshes**


In [ ]:
# Load in the 3D meshes of the radius, ulna, scaphoid, lunate, capitate, and third metacarpal
# Future iterations of this code may incorporate other bones, but carpal alignment quantities are only defined for these bones
# rad_file_path = "/fakepath/L_Radius_Mesh.vtk"
# uln_file_path = "/fakepath/L_Ulna_Mesh.vtk"
# sca_file_path = "/fakepath/L_Scaphoid_Mesh.vtk"
# lun_file_path = "/fakepath/L_Lunate_Mesh.vtk"
# cap_file_path = "/fakepath/L_Capitate_Mesh.vtk"
# mc3_file_path = "/fakepath/L_MC3_Mesh.vtk"
# transform_target_rad_file_path = "/fakepath/Adjusted_L_Radius_Mesh.vtk" # This is only needed if TRANSFORM_RADIUS is set to True
# skin_file_path = "/fakepath/L_Skin_Mesh.vtk" # This is only needed if you want to visualize the skin mesh along with the bones

rad_file_path = "/Volumes/ManskeLab/KRASL/Carpal Alignment Pipeline/Meshes/KRASL_101_R_NEUTRAL_Radius_Mesh.vtk"
uln_file_path = "/Volumes/ManskeLab/KRASL/Carpal Alignment Pipeline/Meshes/KRASL_101_R_NEUTRAL_Ulna_Mesh.vtk"
sca_file_path = "/Volumes/ManskeLab/KRASL/Carpal Alignment Pipeline/Meshes/KRASL_101_R_NEUTRAL_Scaphoid_Mesh.vtk"
lun_file_path = "/Volumes/ManskeLab/KRASL/Carpal Alignment Pipeline/Meshes/KRASL_101_R_NEUTRAL_Lunate_Mesh.vtk"
cap_file_path = "/Volumes/ManskeLab/KRASL/Carpal Alignment Pipeline/Meshes/KRASL_101_R_NEUTRAL_Capitate_Mesh.vtk"
mc3_file_path = "/Volumes/ManskeLab/KRASL/Carpal Alignment Pipeline/Meshes/KRASL_101_R_NEUTRAL_MC3_Mesh.vtk"
transform_target_rad_file_path = "/Volumes/ManskeLab/KRASL/Carpal Alignment Pipeline/Meshes/KRASL_101_PUSHUP_Radius_Mesh.vtk"
skin_file_path = "/Volumes/ManskeLab/KRASL/Carpal Alignment Pipeline/Meshes/KRASL_101_R_NEUTRAL_Skin_Mesh.vtk"

# Indicate the handedness of the meshes being analyzed and the health of the individual
HAND = "R"
TRANSFORM_RADIUS = True # This should be set to true in the case that there is not sufficient length of the radius in one scan, such that the radius from another scan needs to be used for alignment

In [ ]:
# Map the bones to the paths that hold their mesh data (typically in VTK format)
bones = [
    ("Radius", rad_file_path),
    ("Ulna", uln_file_path),
    ("Scaphoid", sca_file_path),
    ("Lunate", lun_file_path),
    ("Capitate", cap_file_path),
    ("MC3", mc3_file_path)
]
skin = pv.read(skin_file_path)

def load_bone_data(path):
    """This function will read in the information about a bone and populate the relevant point and polydata information for the bone.

    Args:
        path (str): File path to the bone mesh in VTK format.
    """
    polydata, points, point_data = read_vtk(path, HAND)
    np_data = vtk_to_numpy(points.GetData())
    vertices, faces = extract_mesh_from_vtk(polydata)
    return polydata, points, point_data, np_data, vertices, faces

# Build a dictionary of the mesh data
krasl_bones = {}
for name, path in bones:
    polydata, points, point_data, np_data, vertices, faces = load_bone_data(path)
    krasl_bones[name] = {
        "Path": path,
        "Polydata": polydata,
        "Points": points,
        "Point_Data": point_data,
        "Numpy_Data": np_data,
        "Vertices": vertices,
        "Faces": faces
    }

# Added a condition to transform the radius to the position of a different scan - this is done for cases where there is a short scan of the radius and the shaft is not visible
if TRANSFORM_RADIUS == True:
    print("Scan did not have full radius length - performing landmark and ICP-based transformation to align radius from other scan to target scan.")
    
    # Read in the original radius mesh and the target radius mesh and process them as was done previously
    radius_polydata, radius_points, radius_point_data = read_vtk(rad_file_path, HAND)
    radius_np_data = vtk_to_numpy(radius_points.GetData())
    radius_vertices, radius_faces = extract_mesh_from_vtk(radius_polydata)
    radius_mesh = pv.PolyData(radius_vertices, radius_faces)
    radius_mesh = radius_mesh.triangulate()
    # Visualizing the target radius mesh
    transform_rad_polydata, transform_rad_points, transform_rad_point_data = read_vtk(transform_target_rad_file_path, HAND)
    transform_rad_np_data = vtk_to_numpy(transform_rad_points.GetData())
    transform_rad_vertices, transform_rad_faces = extract_mesh_from_vtk(transform_rad_polydata)
    transform_rad_mesh = pv.PolyData(transform_rad_vertices, transform_rad_faces)
    transform_rad_mesh = transform_rad_mesh.triangulate()  
    
    cropped_radius_polydata, cropped_transform_rad_polydata = crop_meshes(radius_polydata, transform_rad_polydata)
    landmark_transformed_polydata, landmark_transformed_mesh, landmark_tfm = landmark_transform(cropped_radius_polydata, cropped_transform_rad_polydata)
    icp_radius_polydata, icp_radius_mesh, icp_tfm = icp_transform(cropped_radius_polydata, landmark_transformed_polydata)
    uncropped_transformed_radius_polydata, uncropped_transformed_radius_mesh, combined_tfm = transform_uncropped_mesh(transform_rad_polydata, landmark_tfm, icp_tfm)

    # # The following block of code can be commented out if visualization is not needed
    p = pv.Plotter(window_size = (800, 600))
    p.add_mesh(radius_polydata, color="lightblue", opacity=0.5, label="Cropped Original Radius Mesh")
    p.add_mesh(transform_rad_polydata, color="red", opacity=0.5, label="Cropped Target Radius Mesh")
    # p.add_mesh(landmark_transformed_mesh, color="yellow", opacity=0.5, label="Landmark Transformed Radius Mesh")
    # p.add_mesh(icp_radius_polydata, color="green", opacity=0.5, label="ICP Transformed Radius Mesh")
    p.add_mesh(uncropped_transformed_radius_polydata, color="purple", opacity=0.5, label="Final Transformed Radius Mesh")
    p.show_grid()
    p.show()

    # Updating the radius data in the krasl_bones dictionary to reflect the transformed mesh
    krasl_bones["Radius"]["Polydata"] = uncropped_transformed_radius_polydata
    krasl_bones["Radius"]["Points"] = uncropped_transformed_radius_polydata.GetPoints()
    krasl_bones["Radius"]["Point_Data"] = uncropped_transformed_radius_polydata.GetPointData()
    krasl_bones["Radius"]["Numpy_Data"] = vtk_to_numpy(uncropped_transformed_radius_polydata.GetPoints().GetData())
    krasl_bones["Radius"]["Vertices"], krasl_bones["Radius"]["Faces"] = extract_mesh_from_vtk(uncropped_transformed_radius_polydata)
    
# Perform PCA and derivative calculations for each bone
np_arrays = [krasl_bones[bone]["Numpy_Data"] for bone in krasl_bones]
pca_components, pca_means, pca_variance, component_colours = principal_component_analysis(np_arrays)
first_derivative, second_derivative = [], []
for img in np_arrays:
    first_deriv, second_deriv = compute_derivatives(img)
    first_derivative.append(first_deriv)
    second_derivative.append(second_deriv)

# Storing PCA and derivative results in the krasl_bones dictionary
for i, bone in enumerate(krasl_bones):
    krasl_bones[bone]["PCA_Component_1"] = pca_components[i][0]
    krasl_bones[bone]["PCA_Component_2"] = pca_components[i][1]
    krasl_bones[bone]["PCA_Component_3"] = pca_components[i][2]
    krasl_bones[bone]["PCA_Means"] = pca_means[i]
    krasl_bones[bone]["First Derivatives"] = first_derivative[i]
    krasl_bones[bone]["Second Derivatives"] = second_derivative[i]



##### **Variables used to set the size and shape of the mesh**

These parameters were adjusted to determine the sensitivity of carpal alignment measurements to changes in mesh parameters. The user can choose to adjust these settings to see how carpal alignment measurements change with different inputs.

In [ ]:
# MESH SENSITIVITY PARAMETERS TO ADJUST
CAPITATE_SMOOTHING_ITERATIONS = 60
RADIOCARPAL_SMOOTHING_ITERATIONS = 30
CLUSTERING_POINTS = 10000
SMOOTHING_RELAXATION_FACTOR = 0.10
START_ROW = 269 # Please note that in a CSV file, this will appear in the Excel sheet as 2 greater than this value due to the header row and 0-indexing. For example, if you input 0 here, it will appear in Excel in the B-row.
EXPORT_TO_CSV = False

##### **Mesh Processing**

Process each bone mesh: triangulate, decimate, cluster, smooth, divide into halves based on PCA, and calculate vertex normals and curvatures

In [ ]:
# Mesh Processing

start_time = time.time()

# This block of code will process each bone mesh and calculate the necessary physical properties of the bones
for i, bone in enumerate(krasl_bones):
    # Defining mesh components, including the vertices, faces, and triangulated and clustered polydata
    krasl_bones[bone]["Mesh"] = pv.PolyData(krasl_bones[bone]["Vertices"], krasl_bones[bone]["Faces"])
    krasl_bones[bone]["Mesh"] = krasl_bones[bone]["Mesh"].triangulate()
    clustered_bone = Clustering(krasl_bones[bone]["Mesh"])
    clustered_bone.cluster(CLUSTERING_POINTS) 
    krasl_bones[bone]["Mesh"] = clustered_bone.create_mesh()
    krasl_bones[bone]["Normals"] = calculate_vertex_normals(krasl_bones[bone]["Mesh"])

    # Smoothing the bones based on their morphology to reduce noise and artifacts that may arise from segmentations, which can impact curvature calculations
    if i == 4: # Smoothing the capitate more heavily due to more irregular surfaces in capitate morphology
        krasl_bones[bone]["Mesh"] = smooth_mesh(krasl_bones[bone]["Mesh"], iterations = CAPITATE_SMOOTHING_ITERATIONS, relaxation_factor = SMOOTHING_RELAXATION_FACTOR)
    else:
        krasl_bones[bone]["Mesh"] = smooth_mesh(krasl_bones[bone]["Mesh"], iterations = RADIOCARPAL_SMOOTHING_ITERATIONS, relaxation_factor = SMOOTHING_RELAXATION_FACTOR)
    # Dividing the mesh based on the principal components (from a PCA)
    krasl_bones[bone]["Indices_Above_Mean_PC1"], krasl_bones[bone]["Indices_Below_Mean_PC1"], krasl_bones[bone]["Points_Above_Mean_PC1"], krasl_bones[bone]["Points_Below_Mean_PC1"], krasl_bones[bone]["Colors_PC1"] = half_mesh(krasl_bones[bone]["Mesh"], krasl_bones[bone]["PCA_Means"], krasl_bones[bone]["PCA_Component_1"], is_pca = True)
    krasl_bones[bone]["Indices_Above_Mean_PC2"], krasl_bones[bone]["Indices_Below_Mean_PC2"], krasl_bones[bone]["Points_Above_Mean_PC2"], krasl_bones[bone]["Points_Below_Mean_PC2"], krasl_bones[bone]["Colors_PC2"] = half_mesh(krasl_bones[bone]["Mesh"], krasl_bones[bone]["PCA_Means"], krasl_bones[bone]["PCA_Component_2"], is_pca = True)
    krasl_bones[bone]["Indices_Above_Mean_PC3"], krasl_bones[bone]["Indices_Below_Mean_PC3"], krasl_bones[bone]["Points_Above_Mean_PC3"], krasl_bones[bone]["Points_Below_Mean_PC3"], krasl_bones[bone]["Colors_PC3"] = half_mesh(krasl_bones[bone]["Mesh"], krasl_bones[bone]["PCA_Means"], krasl_bones[bone]["PCA_Component_3"], is_pca = True)


    if HAND == "L": # # When mirroring the left hand scans to become right hands, it is flipping the signs of the curvature and causing positive mean curvatures in pits, which is opposite of what is needed. Therefore, the signs of the curvatures must be flipped.
        krasl_bones[bone]["Normals"] = -krasl_bones[bone]["Normals"]

    # Calculating the curvatures for the radius, scaphoid, and lunate -- these are used in axis definitions.
    if i == 0 or i == 2 or i == 3: # Need to get the curvatures for the radius, scaphoid, and lunate only (for axis definitions)
        krasl_bones[bone]["Curvatures"] = calculate_curvatures(krasl_bones[bone]["Mesh"], krasl_bones[bone]["Normals"], CURVATURE_RADIUS)
        krasl_bones[bone]["Mesh"].point_data["k1"] = krasl_bones[bone]["Curvatures"][:, 0]
        krasl_bones[bone]["Mesh"].point_data["k2"] = krasl_bones[bone]["Curvatures"][:, 1]
        krasl_bones[bone]["Mesh"].point_data["Mean_Curvature"] = krasl_bones[bone]["Curvatures"][:, 2]
        krasl_bones[bone]["Mesh"].point_data["Gaussian_Curvature"] = krasl_bones[bone]["Curvatures"][:, 3]

        krasl_bones[bone]["Normalized_Curvatures"] = normalize_curvatures(krasl_bones[bone]["Curvatures"])
        krasl_bones[bone]["Mesh"].point_data["Normalized_k1"] = krasl_bones[bone]["Normalized_Curvatures"][:, 0]
        krasl_bones[bone]["Mesh"].point_data["Normalized_k2"] = krasl_bones[bone]["Normalized_Curvatures"][:, 1]
        krasl_bones[bone]["Mesh"].point_data["Normalized_Mean_Curvature"] = krasl_bones[bone]["Normalized_Curvatures"][:, 2]
        krasl_bones[bone]["Mesh"].point_data["Normalized_Gaussian_Curvature"] = krasl_bones[bone]["Normalized_Curvatures"][:, 3]

# This helps to track how long the curvature calculations take - if the curvature radius is 5, it typically takes 90 seconds to complete
print("Time to compute curvatures: ", time.time() - start_time, "seconds.")


##### **Optional: Visualize the meshes before proceeding to axis defintions to ensure correct orientations**

Comment out the block of code below if you do not need this visualization.

In [ ]:
# Displaying the PyVista plots of the meshes for visualization
p = pv.Plotter()
for i in range(len(krasl_bones)):
    p.add_mesh(krasl_bones[list(krasl_bones.keys())[i]]["Mesh"], color = "white")
p.show_grid()
p.show()

---
<a name="Carpal-Alignment"></a>
## *Carpal Alignment Definitions*

Defining the values required for axis defintions

In [ ]:
# Radius Information
krasl_bones["Radius"]["Origin"], krasl_bones["Radius"]["Principal_Axis_1"], krasl_bones["Radius"]["Principal_Axis_2"], krasl_bones["Radius"]["Principal_Axis_3"], radial_axis_tube = get_radial_axes(krasl_bones["Radius"], 
                                                                                                                                                                                        krasl_bones["Radius"]["Mesh"], 
                                                                                                                                                                                        krasl_bones["Ulna"], 
                                                                                                                                                                                        shaft_coordinate = RADIUS_SHAFT_DIRECTION,
                                                                                                                                                                                        radial_ulnar_coordinate = RADIAL_ULNAR_DIRECTION, 
                                                                                                                                                                                        palmar_dorsal_coordinate = PALMAR_DORSAL_DIRECTION, 
                                                                                                                                                                                        visualize = True)
radial_axes = np.array(krasl_bones["Radius"]["Principal_Axis_1"]), np.array(krasl_bones["Radius"]["Principal_Axis_2"]), np.array(krasl_bones["Radius"]["Principal_Axis_3"])

# Scaphoid information

# Dividing the scaphoid into sections along its first principal component axis
scaphoid_split_indices, scaphoid_split_points, scaphoid_split_colours = divide_mesh_into_sections(
    krasl_bones["Scaphoid"]["Mesh"],
    krasl_bones["Scaphoid"]["PCA_Means"],
    krasl_bones["Scaphoid"]["PCA_Component_1"],
    divisions = SCAPHOID_DIVISIONS
)
# Adding the scaphoid split data to the krasl_bones dictionary
krasl_bones["Scaphoid"]["Split_Indices"] = scaphoid_split_indices
krasl_bones["Scaphoid"]["Split_Points"] = scaphoid_split_points
krasl_bones["Scaphoid"]["Split_Colours"] = scaphoid_split_colours

adjusted_scaphoid_principal_axes = adjust_scaphoid_principal_axes(krasl_bones["Scaphoid"], radial_axes)
krasl_bones["Scaphoid"]["Centroid"], _ = compute_bone_centroid(krasl_bones["Scaphoid"]["Mesh"])
krasl_bones["Scaphoid"]["Principal_Axis_1"], krasl_bones["Scaphoid"]["Principal_Axis_2"], krasl_bones["Scaphoid"]["Principal_Axis_3"] = adjusted_scaphoid_principal_axes[0], adjusted_scaphoid_principal_axes[1], adjusted_scaphoid_principal_axes[2]
scaphoid_principal_axis_1_arrow = pv.Arrow(start = krasl_bones["Scaphoid"]["Centroid"].GetCenter(), direction = adjusted_scaphoid_principal_axes[0], scale = 15)
scpahoid_principal_axis_2_arrow = pv.Arrow(start = krasl_bones["Scaphoid"]["Centroid"].GetCenter(), direction = adjusted_scaphoid_principal_axes[1], scale = 15)
scaphoid_principal_axis_3_arrow = pv.Arrow(start = krasl_bones["Scaphoid"]["Centroid"].GetCenter(), direction = adjusted_scaphoid_principal_axes[2], scale = 15)


# Lunate information

adjusted_lunate_principal_axes = adjust_lunate_principal_axes(krasl_bones["Lunate"], radial_axes)
krasl_bones["Lunate"]["Centroid"], _ = compute_bone_centroid(krasl_bones["Lunate"]["Mesh"])
krasl_bones["Lunate"]["Principal_Axis_1"], krasl_bones["Lunate"]["Principal_Axis_2"], krasl_bones["Lunate"]["Principal_Axis_3"] = adjusted_lunate_principal_axes[0], adjusted_lunate_principal_axes[1], adjusted_lunate_principal_axes[2]
lunate_principal_axis_1_arrow = pv.Arrow(start = krasl_bones["Lunate"]["Centroid"].GetCenter(), direction = adjusted_lunate_principal_axes[0], scale = 15)
lunate_principal_axis_2_arrow = pv.Arrow(start = krasl_bones["Lunate"]["Centroid"].GetCenter(), direction = adjusted_lunate_principal_axes[1], scale = 15)
lunate_principal_axis_3_arrow = pv.Arrow(start = krasl_bones["Lunate"]["Centroid"].GetCenter(), direction = adjusted_lunate_principal_axes[2], scale = 15)
# Splitting the lunate mesh into two parts based on the first principal inertial axis
krasl_bones["Lunate"]["Indices_Above_Axis_1"], krasl_bones["Lunate"]["Indices_Below_Axis_1"], krasl_bones["Lunate"]["Points_Above_Axis_1"], krasl_bones["Lunate"]["Points_Below_Axis_1"], krasl_bones["Lunate"]["Colors_Axis_1"] = half_mesh(krasl_bones["Lunate"]["Mesh"], krasl_bones["Lunate"]["Centroid"].GetCenter(), adjusted_lunate_principal_axes[0], is_pca = False)
krasl_bones["Lunate"]["Indices_Above_Axis_2"], krasl_bones["Lunate"]["Indices_Below_Axis_2"], krasl_bones["Lunate"]["Points_Above_Axis_2"], krasl_bones["Lunate"]["Points_Below_Axis_2"], krasl_bones["Lunate"]["Colors_Axis_2"] = half_mesh(krasl_bones["Lunate"]["Mesh"], krasl_bones["Lunate"]["Centroid"].GetCenter(), adjusted_lunate_principal_axes[1], is_pca = False)
krasl_bones["Lunate"]["Indices_Above_Axis_3"], krasl_bones["Lunate"]["Indices_Below_Axis_3"], krasl_bones["Lunate"]["Points_Above_Axis_3"], krasl_bones["Lunate"]["Points_Below_Axis_3"], krasl_bones["Lunate"]["Colors_Axis_3"] = half_mesh(krasl_bones["Lunate"]["Mesh"], krasl_bones["Lunate"]["Centroid"].GetCenter(), adjusted_lunate_principal_axes[2], is_pca = False)


# Capitate and MC3 information
krasl_bones["Capitate"]["Centroid"], _ = compute_bone_centroid(krasl_bones["Capitate"]["Mesh"])
krasl_bones["MC3"]["Centroid"], _ = compute_bone_centroid(krasl_bones["MC3"]["Mesh"])

adjusted_mc3_principal_axes = adjust_mc3_principal_axes(krasl_bones["MC3"], radial_axes, krasl_bones["Radius"], position = "NEUTRAL", pa2_flip = False)
krasl_bones["MC3"]["Principal_Axis_1"], krasl_bones["MC3"]["Principal_Axis_2"], krasl_bones["MC3"]["Principal_Axis_3"] = adjusted_mc3_principal_axes[0], adjusted_mc3_principal_axes[1], adjusted_mc3_principal_axes[2]
adjusted_capitate_principal_axes = adjust_capitate_principal_axes(krasl_bones["Capitate"], krasl_bones["MC3"])
krasl_bones["Capitate"]["Principal_Axis_1"], krasl_bones["Capitate"]["Principal_Axis_2"], krasl_bones["Capitate"]["Principal_Axis_3"] = adjusted_capitate_principal_axes[0], adjusted_capitate_principal_axes[1], adjusted_capitate_principal_axes[2]

# Optional: Displaying all of the carpal bones and their principal axes for visualization
# Comment the following block out if you do not wish to see this visualization

p = pv.Plotter()
for i in range(len(krasl_bones)):
    p.add_mesh(krasl_bones[list(krasl_bones.keys())[i]]["Mesh"], color = "white", opacity = 0.5)
for i, bone in enumerate(krasl_bones):
    if i == 0:
        p.add_mesh(pv.Arrow(start = krasl_bones[bone]["Origin"].GetCenter(), direction = krasl_bones[bone]["Principal_Axis_1"], scale = 10), color = "red")
        p.add_mesh(pv.Arrow(start = krasl_bones[bone]["Origin"].GetCenter(), direction = krasl_bones[bone]["Principal_Axis_2"], scale = 10), color = "green")
        p.add_mesh(pv.Arrow(start = krasl_bones[bone]["Origin"].GetCenter(), direction = krasl_bones[bone]["Principal_Axis_3"], scale = 10), color = "blue")
    elif i >= 2:
        p.add_mesh(pv.Arrow(start = krasl_bones[bone]["Centroid"].GetCenter(), direction = krasl_bones[bone]["Principal_Axis_1"], scale = 10), color = "red")
        p.add_mesh(pv.Arrow(start = krasl_bones[bone]["Centroid"].GetCenter(), direction = krasl_bones[bone]["Principal_Axis_2"], scale = 10), color = "green")
        p.add_mesh(pv.Arrow(start = krasl_bones[bone]["Centroid"].GetCenter(), direction = krasl_bones[bone]["Principal_Axis_3"], scale = 10), color = "blue")
p.show_grid()
p.show()






##### **Defining the sagittal and anteroposterior planes that the 3D axes and carpal bone meshes will be projected onto**

In [ ]:
radius_data = krasl_bones["Radius"]
ulna_data = krasl_bones["Ulna"]
scaphoid_data = krasl_bones["Scaphoid"]
lunate_data = krasl_bones["Lunate"]
capitate_data = krasl_bones["Capitate"]
mc3_data = krasl_bones["MC3"]

# Creating arrows for the radius axes to be used in PyVista visualizations
radius_axes = np.array([radius_data["Principal_Axis_1"], radius_data["Principal_Axis_2"], radius_data["Principal_Axis_3"]])
radius_x_axis_arrow, radius_y_axis_arrow, radius_z_axis_arrow = pv.Arrow(start = radius_data["Origin"].GetCenter(), direction = radius_data["Principal_Axis_1"], scale = 15), pv.Arrow(start = radius_data["Origin"].GetCenter(), direction = radius_data["Principal_Axis_2"], scale = 15), pv.Arrow(start = radius_data["Origin"].GetCenter(), direction = radius_data["Principal_Axis_3"], scale = 15)

# Creating planes aligned along the x-axis and z-axis of the radius and starts at the radius origin and 2D visualizations of the carpal bones
radial_plane_origin = radius_data["Origin"].GetCenter()
ap_plane_normal = radius_data["Principal_Axis_3"]
ap_plane_normal = ap_plane_normal / np.linalg.norm(ap_plane_normal)
ap_plane = pv.Plane(center = radial_plane_origin, direction = ap_plane_normal, i_size = 100, j_size = 100)
camera_position_ap = radial_plane_origin + ap_plane_normal * SCENE_DISTANCE
sagittal_plane_normal = radius_data["Principal_Axis_2"]
sagittal_plane_normal = sagittal_plane_normal / np.linalg.norm(sagittal_plane_normal)
sagittal_plane = pv.Plane(center = radial_plane_origin, direction = sagittal_plane_normal, i_size = 100, j_size = 100)
camera_position_sagittal = radial_plane_origin + sagittal_plane_normal * SCENE_DISTANCE
view_up_vector = np.array([0, 0, 1])

# Projecting the 3D meshes onto the ap and sagittal planes for visualization
radius_projected_ap = radius_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
scaphoid_projected_ap = scaphoid_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
lunate_projected_ap = lunate_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
capitate_projected_ap = capitate_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
mc3_projected_ap = mc3_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)

radius_projected_sagittal = radius_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
scaphoid_projected_sagittal = scaphoid_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
lunate_projected_sagittal = lunate_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
capitate_projected_sagittal = capitate_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
mc3_projected_sagittal = mc3_data["Mesh"].project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)

# Optional visualizations below of the projected meshes in the AP and sagittal planes

# # Visualizing the projected meshes in the ap plane
# p = pv.Plotter()
# p.add_mesh(ap_plane, style = "wireframe", color = "lightgrey", opacity = 0.5)
# p.add_mesh(radius_projected_ap, color = "red", opacity = 0.5)
# p.add_mesh(scaphoid_projected_sagittal_ap, color = "green", opacity = 0.5)
# p.add_mesh(lunate_projected_sagittal_ap, color = "yellow", opacity = 0.5)
# p.add_mesh(capitate_projected_ap, color = "purple", opacity = 0.5)
# p.add_mesh(mc3_projected_ap, color = "orange", opacity = 0.5)
# p.camera_position = [camera_position_ap, radial_plane_origin, view_up_vector]
# p.show()

# # Visualizing the projected meshes in the sagittal plane
# p = pv.Plotter()
# p.add_mesh(sagittal_plane, style = "wireframe", color = "lightgrey", opacity = 0.5)
# p.add_mesh(radius_projected_sagittal, color = "red", opacity = 0.5)
# p.add_mesh(scaphoid_projected_sagittal_sagittal, color = "green", opacity = 0.5)
# p.add_mesh(lunate_projected_sagittal_sagittal, color = "yellow", opacity = 0.5)
# p.add_mesh(capitate_projected_sagittal, color = "purple", opacity = 0.5)
# p.add_mesh(mc3_projected_sagittal, color = "orange", opacity = 0.5)
# p.camera_position = [camera_position_sagittal, radial_plane_origin, view_up_vector]
# p.show()


##### **Carpal Alignment Axis Definitions - 3D, sagittal projection, and anteroposterior projection**

The following measurements are included in the block below:
- Scaphoid axis
- Lunate axis
- Capitate axis
- MC3 axis
- Capitate-Radius Index distance measurement
- Scapholunate Interval measurements
- Dorsal scaphoid translation measurements

In [ ]:
# Defining the axes for the scaphoid, lunate, capitate, and MC3

# Defining the 3D vectors and lines that represent the axes of each bone
scaphoid_line, scaphoid_tube, scaphoid_vector, scaphoid_normalized_vector, scaphoid_arrow, scaphoid_intersection_points = get_scaphoid_axis(scaphoid_data, visualize = True)
lunate_guide_line, lunate_guide_vector, lunate_tube, lunate_vector = get_lunate_axis(lunate_data, radius_axes, visualize = True)
capitate_tube, capitate_vector = get_capitate_axis(capitate_data, visualize = True)
mc3_tube, mc3_vector = get_mc3_axis(mc3_data, visualize = True)

# Defining the 2D vectors and lines used in the official measurements of carpal alignment
# Sagittal Plane Projections
radius_2d_sag_line_projected = radial_axis_tube.project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
radius_2d_sag_vector_projected = np.array(radius_2d_sag_line_projected.points[0]) - np.array(radius_2d_sag_line_projected.points[-1])
radius_2d_sag_vector_projected = radius_2d_sag_vector_projected / np.linalg.norm(radius_2d_sag_vector_projected)

scaphoid_2d_sag_line_projected = scaphoid_tube.project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
sca_2d_sag_vector = scaphoid_vector - np.dot(scaphoid_vector, sagittal_plane_normal) * sagittal_plane_normal
sca_2d_sag_vector = sca_2d_sag_vector / np.linalg.norm(sca_2d_sag_vector)
sca_2d_sag_vector_arrow = pv.Arrow(start = scaphoid_2d_sag_line_projected.points[0], direction = sca_2d_sag_vector, scale = 15)

lunate_2d_sag_line_projected = lunate_tube.project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
lunate_2d_sag_guideline_projected = lunate_guide_line.project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
lun_2d_sag_vector = lunate_vector - np.dot(lunate_vector, sagittal_plane_normal) * sagittal_plane_normal
lun_2d_sag_vector = lun_2d_sag_vector / np.linalg.norm(lun_2d_sag_vector)
lun_2d_sag_vector_arrow = pv.Arrow(start = lunate_2d_sag_line_projected.points[0], direction = lun_2d_sag_vector, scale = 15)

capitate_2d_sag_line_projected = capitate_tube.project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
cap_2d_sag_vector = capitate_vector - np.dot(capitate_vector, sagittal_plane_normal) * sagittal_plane_normal
cap_2d_sag_vector = cap_2d_sag_vector / np.linalg.norm(cap_2d_sag_vector)
cap_2d_sag_vector_arrow = pv.Arrow(start = capitate_2d_sag_line_projected.points[0], direction = cap_2d_sag_vector, scale = 15)

mc3_2d_sag_line_projected = mc3_tube.project_points_to_plane(origin = radial_plane_origin, normal = sagittal_plane_normal)
mc3_2d_sag_vector = mc3_vector - np.dot(mc3_vector, sagittal_plane_normal) * sagittal_plane_normal
mc3_2d_sag_vector = mc3_2d_sag_vector / np.linalg.norm(mc3_2d_sag_vector)
mc3_2d_vector_arrow = pv.Arrow(start = mc3_2d_sag_line_projected.points[0], direction = mc3_2d_sag_vector, scale = 15)


# AP Projections
radius_2d_ap_line_projected = radial_axis_tube.project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
radius_2d_ap_vector_projected = np.array(radius_2d_ap_line_projected.points[0]) - np.array(radius_2d_ap_line_projected.points[-1])
radius_2d_ap_vector_projected = radius_2d_ap_vector_projected / np.linalg.norm(radius_2d_ap_vector_projected)

scaphoid_2d_ap_line_projected = scaphoid_tube.project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
sca_2d_ap_vector = scaphoid_vector - np.dot(scaphoid_vector, ap_plane_normal) * ap_plane_normal
sca_2d_ap_vector = sca_2d_ap_vector / np.linalg.norm(sca_2d_ap_vector)
sca_2d_ap_vector_arrow = pv.Arrow(start = scaphoid_2d_ap_line_projected.points[0], direction = sca_2d_ap_vector, scale = 15)

lunate_2d_ap_line_projected = lunate_tube.project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
lunate_2d_ap_guideline_projected = lunate_guide_line.project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
lun_2d_ap_vector = lunate_vector - np.dot(lunate_vector, ap_plane_normal) * ap_plane_normal
lun_2d_ap_vector = lun_2d_ap_vector / np.linalg.norm(lun_2d_ap_vector)
lun_2d_ap_vector_arrow = pv.Arrow(start = lunate_2d_ap_line_projected.points[0], direction = lun_2d_ap_vector, scale = 15)

capitate_2d_ap_line_projected = capitate_tube.project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
cap_2d_ap_vector = capitate_vector - np.dot(capitate_vector, ap_plane_normal) * ap_plane_normal
cap_2d_ap_vector = cap_2d_ap_vector / np.linalg.norm(cap_2d_ap_vector)
cap_2d_ap_vector_arrow = pv.Arrow(start = capitate_2d_ap_line_projected.points[0], direction = cap_2d_ap_vector, scale = 15)

mc3_2d_ap_line_projected = mc3_tube.project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
mc3_2d_ap_vector = mc3_vector - np.dot(mc3_vector, ap_plane_normal) * ap_plane_normal
mc3_2d_ap_vector = mc3_2d_ap_vector / np.linalg.norm(mc3_2d_ap_vector)
mc3_2d_vector_arrow = pv.Arrow(start = mc3_2d_ap_line_projected.points[0], direction = mc3_2d_ap_vector, scale = 15)



# Calculating the distance-based carpal alignment measurements
# This includes the capitate-radius index, the scapholunate intervals, and dorsal scaphoid translation

# Capitate-Radius Index
capitate_radius_distance, capitate_radius_line, cri_meshes_list = nearest_neighbour_distance(capitate_data["Mesh"], radius_data["Mesh"])
capitate_radius_line_projected_ap = capitate_radius_line.project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)

# Scapholunate Intervals
min_scapholunate_interval, scapholunate_line, sli_meshes_list = nearest_neighbour_distance(scaphoid_data["Mesh"], lunate_data["Mesh"])
min_scapholunate_interval_line_projected_xy = scapholunate_line.project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
articulation_threshold = np.max([SL_ARTICULATION_THRESHOLD, min_scapholunate_interval * 2.5])
print("Articulation Threshold: ", articulation_threshold)
_, _, lunate_surface_mesh, _, _, _, _, lunate_midpoints, _, lunate_min_distance_lines, lunate_distances, positions, _, _ = define_articulating_surface(lunate_data, scaphoid_data, articulation_threshold, MIN_SL_ANGLE, MAX_SL_ANGLE)
lunate_surface_mesh_points = lunate_surface_mesh.points
lunate_surface_mesh_center = pv.PolyData(lunate_surface_mesh_points.mean(axis=0).reshape(1, 3))

proximal_sl_line_2D = lunate_min_distance_lines[0].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
palmar_sl_line_2D = lunate_min_distance_lines[1].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
distal_sl_line_2D = lunate_min_distance_lines[2].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
dorsal_sl_line_2D = lunate_min_distance_lines[3].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
central_sl_line_2D = lunate_min_distance_lines[4].project_points_to_plane(origin = radial_plane_origin, normal = ap_plane_normal)
proximal_sl_interval = lunate_distances[0]
palmar_sl_interval = lunate_distances[1]
distal_sl_interval = lunate_distances[2]
dorsal_sl_interval = lunate_distances[3]
central_sl_interval = lunate_distances[4]

# Dorsal Scaphoid Translation
# Dorsal Tangential Line (DTL) Method
dtl_dst_components = get_3D_dst_components(radius_data, radius_axes, scaphoid_data, "DTL")
scaphoid_dorsal_point, scaphoid_dorsal_line, scaphoid_dorsal_tube, dorsal_radius_point, radius_rim_line, radius_rim_tube, max_points = dtl_dst_components[0], dtl_dst_components[1], dtl_dst_components[2], dtl_dst_components[3], dtl_dst_components[4], dtl_dst_components[5], dtl_dst_components[6]
scaphoid_dorsal_line_projected, radius_rim_line_projected, line_midpoint_pv, z_axis_projected, intersection_point, dorsal_scaphoid_translation = measure_dtl_dst(scaphoid_dorsal_line, radius_rim_line, radius_axes, radius_data["Origin"].GetCenter(), sagittal_plane_normal)

# Posterior Radioscaphoid Angle (PRSA) Method
prsa_dst_components = get_3D_dst_components(radius_data, radius_axes, scaphoid_data, "PRSA")
dorsalRadius_to_dorsalScaphoid_tube, palmarRadius_to_dorsalRadius_tube, dorsal_radius_point, palmar_radius_point, scaphoid_dorsal_point = prsa_dst_components[0], prsa_dst_components[1], prsa_dst_components[2], prsa_dst_components[3], prsa_dst_components[4]
dst_sagittal_plane, slice_origin, radius_slice, scaphoid_slice = prsa_dst_components[5], prsa_dst_components[6], prsa_dst_components[7], prsa_dst_components[8]
dorsal_line_projected, radius_line_projected, dorsal_vector_projected, radius_vector_projected = prsa_dst_components[9], prsa_dst_components[10], prsa_dst_components[11], prsa_dst_components[12]
prsa_3D_angle, prsa_2D_angle, dorsal_rs_line_projected, radius_line_projected, dorsal_rs_line_vector_projected, radius_line_vector_projected = measure_prsa_dst(radius_data, radius_axes, scaphoid_data, dorsalRadius_to_dorsalScaphoid_tube, palmarRadius_to_dorsalRadius_tube, dst_sagittal_plane)

# Optional: Display the DTL lines and meshes for visualization
# Comment this block out if you do not wish to see this visualization

# p = pv.Plotter(window_size = [600, 600])
# p.add_mesh(radius_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 1)
# p.add_mesh(ulna_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.1)
# p.add_mesh(scaphoid_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.8)
# p.add_mesh(lunate_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.1)
# p.add_mesh(capitate_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.1)
# p.add_mesh(mc3_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.1)
# p.add_mesh(scaphoid_dorsal_point, color = "red", point_size = 3)
# p.add_mesh(scaphoid_dorsal_tube, color = "red", point_size = 10)
# p.add_mesh(dorsal_radius_point, color = "blue", point_size = 10)
# p.add_mesh(radius_rim_tube, color = "black")
# camera_position = "zy"
# p.camera_position = camera_position
# p.show_grid()
# p.show()





---

# *Results*

In [ ]:
# Calculating the carpal alignment measurements
print("Mesh Parameters:")
print("Capitate Smoothing Iterations: ", CAPITATE_SMOOTHING_ITERATIONS)
print("Radiocarpal Smoothing Iterations: ", RADIOCARPAL_SMOOTHING_ITERATIONS)
print("Smoothing Relaxation Rate: ", SMOOTHING_RELAXATION_FACTOR)
print("Mesh Number of Points: ", CLUSTERING_POINTS)
print("--------------------")

# Scapholunate Angle
scapholunate_3d_angle = calculate_3D_angle(scaphoid_vector, lunate_vector)
scapholunate_2d_sag_angle = calculate_2D_angle(sca_2d_sag_vector, lun_2d_sag_vector)
scapholunate_2d_ap_angle = calculate_2D_angle(sca_2d_ap_vector, lun_2d_ap_vector)
print("3D Scapholunate Angle: ", round(scapholunate_3d_angle, 4))
print("2D Sagittal Scapholunate Angle: ", round(scapholunate_2d_sag_angle, 4))
print("2D AP Scapholunate Angle: ", round(scapholunate_2d_ap_angle, 4))
print("--------------------")

# Radioscaphoid Angle
radioscaphoid_3d_angle = calculate_3D_angle(krasl_bones["Radius"]["Principal_Axis_1"], scaphoid_vector)
radioscaphoid_2d_sag_angle = calculate_2D_angle(radius_2d_sag_vector_projected, sca_2d_sag_vector)
radioscaphoid_2d_ap_angle = calculate_2D_angle(radius_2d_ap_vector_projected, sca_2d_ap_vector)
print("3D Radioscaphoid Angle: ", round(radioscaphoid_3d_angle, 4))
print("2D Sagittal Radioscaphoid Angle: ", round(radioscaphoid_2d_sag_angle, 4))
print("2D AP Radioscaphoid Angle: ", round(radioscaphoid_2d_ap_angle, 4))
print("--------------------")

# Radiolunate Angle
radiolunate_3d_angle = calculate_3D_angle(krasl_bones["Radius"]["Principal_Axis_1"], lunate_vector)
radiolunate_2d_sag_angle = calculate_2D_angle(radius_2d_sag_vector_projected, lun_2d_sag_vector)
radiolunate_2d_ap_angle = calculate_2D_angle(radius_2d_ap_vector_projected, lun_2d_ap_vector)
print("3D Radiolunate Angle: ", round(radiolunate_3d_angle, 4))
print("2D Sagittal Radiolunate Angle: ", round(radiolunate_2d_sag_angle, 4))
print("2D AP Radiolunate Angle: ", round(radiolunate_2d_ap_angle, 4))
print("--------------------")

# Radiocapitate Angle
radiocapitate_3d_angle = calculate_3D_angle(krasl_bones["Radius"]["Principal_Axis_1"], capitate_vector)
radiocapitate_2d_sag_angle = calculate_2D_angle(radius_2d_sag_vector_projected, cap_2d_sag_vector)
radiocapitate_2d_ap_angle = calculate_2D_angle(radius_2d_ap_vector_projected, cap_2d_ap_vector)
print("3D Radiocapitate Angle: ", round(radiocapitate_3d_angle, 4))
print("2D Sagittal Radiocapitate Angle: ", round(radiocapitate_2d_sag_angle, 4))
print("2D AP Radiocapitate Angle: ", round(radiocapitate_2d_ap_angle, 4))
print("--------------------")

# Capitate-MC3 Angle
capitate_mc3_3d_angle = calculate_3D_angle(capitate_vector, mc3_vector)
capitate_mc3_2d_sag_angle = calculate_2D_angle(cap_2d_sag_vector, mc3_2d_sag_vector)
capitate_mc3_2d_ap_angle = calculate_2D_angle(cap_2d_ap_vector, mc3_2d_ap_vector)
print("3D Capitate-MC3 Angle: ", round(capitate_mc3_3d_angle, 4))
print("2D Sagittal Capitate-MC3 Angle: ", round(capitate_mc3_2d_sag_angle, 4))
print("2D AP Capitate-MC3 Angle: ", round(capitate_mc3_2d_ap_angle, 4))
print("--------------------")

# Capitolunate Angle
capitolunate_3d_angle = calculate_3D_angle(capitate_vector, lunate_vector)
capitolunate_2d_sag_angle = calculate_2D_angle(cap_2d_sag_vector, lun_2d_sag_vector)
capitolunate_2d_ap_angle = calculate_2D_angle(cap_2d_ap_vector, lun_2d_ap_vector)
print("3D Capitolunate Angle: ", round(capitolunate_3d_angle, 4))
print("2D Sagittal Capitolunate Angle: ", round(capitolunate_2d_sag_angle, 4))
print("2D AP Capitolunate Angle: ", round(capitolunate_2d_ap_angle, 4))
print("--------------------")

# Capitate-Radius Index
print("Capitate-Radius Index: ", round(capitate_radius_distance / 10.0, 4), "cm.")
print("2D Capiate-Radius Index: ", round(capitate_radius_line.length / 10.0, 4), "cm.")
print("--------------------")

# Scapholunate Intervals
print("Minimum Scapholunate Interval: ", round(min_scapholunate_interval, 2), "mm.")
for i in range(len(positions)):
    print("{} SL Interval: {} mm.".format(positions[i], round(lunate_distances[i], 2)))
print("--------------------")
# print("2D SL Interval Measurements:")
# print("2D Minimum SL Interval: ", round(min_scapholunate_interval_line_projected_xy.length, 2), "mm.")
# print("2D Proximal SL Interval: ", round(proximal_sl_line_2D.length, 2), "mm.")
# print("2D Palmar SL Interval: ", round(palmar_sl_line_2D.length, 2), "mm.")
# print("2D Distal SL Interval: ", round(distal_sl_line_2D.length, 2), "mm.")
# print("2D Dorsal SL Interval: ", round(dorsal_sl_line_2D.length, 2), "mm.")
# print("2D Central SL Interval: ", round(central_sl_line_2D.length, 2), "mm.")
# print("--------------------")

# Dorsal Scaphoid Translation
print("DTL Dorsal Scaphoid Translation: ", round(dorsal_scaphoid_translation, 2), "mm.")
print("PRSA 3D Angle: ", round(prsa_3D_angle, 2), "degrees.")
print("PRSA 2D Angle: ", round(prsa_2D_angle, 2), "degrees.")



---

##### **Optional: Export results**

This block of code provides the option to export your results to a CSV file.

In [ ]:
# Exporting results to a CSV file
csv_path = "/Users/justensaini/Desktop/KRASL_Results_Spreadsheets/KRASL_Carpal_Alignment_Mesh_Sensitivity_Feb2.csv"
# Add two error conditions here: 
# 1) if the csv file does not already exists, create a new one with the appropriate headers
# 2) if the csv file is the wrong size, print an error message and exit the function
if not os.path.exists(csv_path):
    headers = ["Patient_ID", "Position",
               "3D_Scapholunate_Angle", "2D_Sagittal_Scapholunate_Angle", "2D_AP_Scapholunate_Angle",
               "3D_Radioscaphoid_Angle", "2D_Sagittal_Radioscaphoid_Angle", "2D_AP_Radioscaphoid_Angle",
               "3D_Radiolunate_Angle", "2D_Sagittal_Radiolunate_Angle", "2D_AP_Radiolunate_Angle",
               "3D_Radiocapitate_Angle", "2D_Sagittal_Radiocapitate_Angle", "2D_AP_Radiocapitate_Angle",
               "3D_Capitate_MC3_Angle", "2D_Sagittal_Capitate_MC3_Angle", "2D_AP_Capitate_MC3_Angle",
               "3D_Capitolunate_Angle", "2D_Sagittal_Capitolunate_Angle", "2D_AP_Capitolunate_Angle",
               "Capitate_Radius_Index_cm", "2D_Capitate_Radius_Index_cm",
               "Minimum_Scapholunate_Interval_mm", "Proximal_SL_Interval_mm", "Palmar_SL_Interval_mm",
               "Distal_SL_Interval_mm", "Dorsal_SL_Interval_mm",  "Central_SL_Interval_mm",
               "DTL_Dorsal_Scaphoid_Translation_mm", "PRSA_3D_Angle_degrees", "PRSA_2D_Angle_degrees"]
    df = pd.DataFrame(columns = headers)
    df.to_csv(csv_path, index = False)
    print("CSV file created at: ", csv_path)
# If the CSV file exists, check if it has the correct number of columns

df = pd.read_csv(csv_path)
expected_num_columns = 29
if df.shape[1] != expected_num_columns:
    print("Error: The CSV file at {} has {} columns, but {} columns are expected.".format(csv_path, df.shape[1], expected_num_columns))
    print("Please check the CSV file format.")
    exit()

start_column = 9
start_row = START_ROW # Please note that in a CSV file, this will appear in the Excel sheet as 2 greater than this value due to the header row and 0-indexing. For example, if you input 0 here, it will appear in Excel in the B-row.
patient_id = "KRASL_101"
position = "NEUTRAL"
values = [round(scapholunate_3d_angle, 4), round(scapholunate_2d_sag_angle, 4), round(scapholunate_2d_ap_angle, 4),
            round(radioscaphoid_3d_angle, 4), round(radioscaphoid_2d_sag_angle, 4), round(radioscaphoid_2d_ap_angle, 4),
            round(radiolunate_3d_angle, 4), round(radiolunate_2d_sag_angle, 4), round(radiolunate_2d_ap_angle, 4),
            round(radiocapitate_3d_angle, 4), round(radiocapitate_2d_sag_angle, 4), round(radiocapitate_2d_ap_angle, 4),
            round(capitate_mc3_3d_angle, 4), round(capitate_mc3_2d_sag_angle, 4), round(capitate_mc3_2d_ap_angle, 4),
            round(capitolunate_3d_angle, 4), round(capitolunate_2d_sag_angle, 4), round(capitolunate_2d_ap_angle, 4),
            round(capitate_radius_distance / 10.0, 4), round(capitate_radius_line.length / 10.0, 4),
            round(min_scapholunate_interval, 2), round(proximal_sl_interval, 2), round(palmar_sl_interval, 2),
            round(distal_sl_interval, 2), round(dorsal_sl_interval, 2), round(central_sl_interval, 2),
            round(dorsal_scaphoid_translation, 2), round(prsa_3D_angle, 2), round(prsa_2D_angle, 2)]

def export_to_csv(csv_path, start_column, start_row, patient_id, position, values):
    """This function will populate a CSV file with the carpal alignment measurements.

    Args:
        csv_path (str): The path to the CSV file.
        start_column (int): The starting column index for the measurements.
        start_row (int): The starting row index for the measurements.
        patient_id (str): The patient identifier.
        position (str): The wrist position (in the case that you are evaluating multiple positions)
        group (str): The patient group (e.g., Healthy, Pathologic, etc.)
        values (list): The list of carpal alignment measurement values to be recorded.
    """
    df = pd.read_csv(csv_path)
    if start_row is None:
        row_index = df[(df['Patient_ID'] == patient_id) & (df['Position'] == position)].index
        row = row_index[0]
        if len(row_index) == 0:
            print("Patient ID and Position combination not found in CSV. Please check the inputs.")
    else:
        row = start_row
    print("Inserting data at column index: ", start_column - 1, " and row index: ", row)
    df.iloc[row, (start_column-1):(start_column-1) + len(values)] = values
    df.to_csv(csv_path, index = False)
    print("Carpal alignment measurements exported to CSV successfully.")

# Comment the line out below if you want to run the overall code but do not want to export to a CSV file
if EXPORT_TO_CSV == True:
    export_to_csv(csv_path, start_column, start_row, patient_id = patient_id, position = position, values = values)



---

##### **Results visualization**

In [ ]:
# Presenting all PyVista measurements in a large set of plots

# --------------- SCAPHOLUNATE ANGLE ---------------
p = pv.Plotter(shape = (9, 3), window_size = (1400, 4000))

p.subplot(0, 0)
p.add_text("3D Scapholunate Angle", font_size = 12, position = "lower_left", color = "black")
p.add_text("3D SL Angle: {:.2f} °".format(scapholunate_3d_angle), font_size = 10, position = "lower_right", color = "black")
for i, bone in enumerate(krasl_bones):
    p.add_mesh(krasl_bones[bone]["Mesh"], color="white", show_scalar_bar = False, opacity = 0.25)
p.add_mesh(scaphoid_tube, color = "red")
p.add_mesh(lunate_guide_line, color = "grey")
p.add_mesh(lunate_tube, color = "red")
p.add_mesh(lunate_data["Mesh"], color = "yellow", opacity = 0.8)
p.add_mesh(scaphoid_data["Mesh"], color = "green", opacity = 0.8)
p.camera_position = "xy"
elevation = 20

p.subplot(0, 1)
p.add_text("2D Scapholunate Angle\n(Simulated Sagittal X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D SL Sagittal Angle: {:.2f} °".format(scapholunate_2d_sag_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(scaphoid_2d_sag_line_projected, color = "red")
p.add_mesh(lunate_2d_sag_line_projected, color = "red")
p.add_mesh(lunate_2d_sag_guideline_projected, color = "grey")
p.add_mesh(scaphoid_projected_sagittal, color = "green", show_scalar_bar = False, opacity = 0.5)
p.add_mesh(lunate_projected_sagittal, color = "yellow", show_scalar_bar = False, opacity = 0.5)
p.add_mesh(sca_2d_sag_vector_arrow, color = "red")
p.add_mesh(lun_2d_sag_vector_arrow, color = "yellow")
p.camera_position, p.camera.roll, p.camera.azimuth = [camera_position_sagittal, radial_plane_origin, view_up_vector], 0, 180    

p.subplot(0, 2)
p.add_text("2D Scapholunate Angle\n(Simulated AP X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D SL AP Angle: {:.2f} °".format(scapholunate_2d_ap_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(scaphoid_2d_ap_line_projected, color = "red")
p.add_mesh(lunate_2d_ap_line_projected, color = "red")
p.add_mesh(lunate_2d_ap_guideline_projected, color = "grey")
p.add_mesh(scaphoid_projected_ap, color = "green", show_scalar_bar = False, opacity = 0.5)
p.add_mesh(lunate_projected_ap, color = "yellow", show_scalar_bar = False, opacity = 0.5)
p.camera_position, p.camera.roll = [camera_position_ap, radial_plane_origin, view_up_vector], 0


# --------------- RADIOSCAPHOID ANGLE ---------------
p.subplot(1, 0)
p.add_text("3D Radioscaphoid Angle", font_size = 12, position = "lower_left", color = "black")
p.add_text("3D RS Angle: {:.2f} °".format(radioscaphoid_3d_angle), font_size = 10, position = "lower_right", color = "black")
for i, bone in enumerate(krasl_bones):
    if i == 0:
        p.add_mesh(krasl_bones[bone]["Mesh"], color="white", show_scalar_bar = False, opacity = 0)
    else:
        p.add_mesh(krasl_bones[bone]["Mesh"], color="white", show_scalar_bar = False, opacity = 0.25)
p.add_mesh(krasl_bones["Scaphoid"]["Mesh"], color = "green", opacity = 1)
p.add_mesh(radius_x_axis_arrow, color = "red")
p.add_mesh(radius_y_axis_arrow, color = "green")
p.add_mesh(radius_z_axis_arrow, color = "blue")
p.add_mesh(radial_axis_tube, color = "red")
p.add_mesh(radial_axis_tube, color = "red")
p.add_mesh(scaphoid_tube, color = "red")
p.camera_position = "xy"

p.subplot(1, 1)
p.add_text("2D Radioscaphoid Angle\n(Simulated Sagittal X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D RS Sagittal Angle: {:.2f} °".format(radioscaphoid_2d_sag_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(scaphoid_2d_sag_line_projected, color = "red")
p.add_mesh(radius_2d_sag_line_projected, color = "red")
p.add_mesh(scaphoid_projected_sagittal, color = "green", show_scalar_bar = False, opacity = 0.1)
p.camera_position, p.camera.roll, p.camera.azimuth = [camera_position_sagittal, radial_plane_origin, view_up_vector], 0, 180

p.subplot(1, 2)
p.add_text("2D Radioscaphoid Angle\n(Simulated AP X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D RS AP Angle: {:.2f} °".format(radioscaphoid_2d_ap_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(radius_2d_ap_line_projected, color = "red")
p.add_mesh(scaphoid_2d_ap_line_projected, color = "red")
p.add_mesh(radius_projected_ap, color = "red", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(scaphoid_projected_ap, color = "green", show_scalar_bar = False, opacity = 0.1)
p.camera_position, p.camera.roll = [camera_position_ap, radial_plane_origin, view_up_vector], 0


# --------------- RADIOULNATE ANGLE ---------------
p.subplot(2, 0)
p.add_text("3D Radiolunate Angle", font_size = 12, position = "lower_left", color = "black")
p.add_text("3D RL Angle: {:.2f} °".format(radiolunate_3d_angle), font_size = 10, position = "lower_right", color = "black")
for i, bone in enumerate(krasl_bones):
    p.add_mesh(krasl_bones[bone]["Mesh"], color="white", show_scalar_bar = False, opacity = 0.25)
p.add_mesh(krasl_bones["Radius"]["Mesh"], color = "red", opacity = 1)
p.add_mesh(krasl_bones["Lunate"]["Mesh"], color = "yellow", opacity = 1)
p.add_mesh(radius_x_axis_arrow, color = "red")
p.add_mesh(radius_y_axis_arrow, color = "green")
p.add_mesh(radius_z_axis_arrow, color = "blue")
p.add_mesh(radial_axis_tube, color = "red")
p.add_mesh(lunate_guide_line, color = "grey")
p.add_mesh(lunate_tube, color = "red")
p.camera_position = "xy"

p.subplot(2, 1)
p.add_text("2D Radiolunate Angle\n(Simulated Sagittal X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D RL Sagittal Angle: {:.2f} °".format(radiolunate_2d_sag_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(lunate_2d_sag_guideline_projected, color = "grey")
p.add_mesh(lunate_2d_sag_line_projected, color = "red")
p.add_mesh(radius_2d_sag_line_projected, color = "red")
p.add_mesh(lunate_projected_sagittal, color = "orange", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(radius_projected_sagittal, color = "blue", show_scalar_bar = False, opacity = 0.1)
p.camera_position, p.camera.roll, p.camera.azimuth = [camera_position_sagittal, radial_plane_origin, view_up_vector], 0, 180

p.subplot(2, 2)
p.add_text("2D Radiolunate Angle\n(Simulated AP X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D RL AP Angle: {:.2f} °".format(radiolunate_2d_ap_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(lunate_2d_ap_guideline_projected, color = "grey")
p.add_mesh(lunate_2d_ap_line_projected, color = "red")
p.add_mesh(radius_2d_ap_line_projected, color = "red")
p.add_mesh(lunate_projected_ap, color = "yellow", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(radius_projected_ap, color = "red", show_scalar_bar = False, opacity = 0.1)
p.camera_position, p.camera.roll = [camera_position_ap, radial_plane_origin, view_up_vector], 0


# --------------- RADIOCAPITATE ANGLE ---------------
p.subplot(3, 0)
p.add_text("3D Radiocapitate Angle (Capitate Axis)", font_size = 12, position = "lower_left", color = "black")
p.add_text("3D RC Angle: {:.2f} °".format(radiocapitate_3d_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(krasl_bones["Radius"]["Mesh"], color = "red", opacity = 0.75)
p.add_mesh(krasl_bones["Capitate"]["Mesh"], color = "purple", opacity = 0.75)
p.add_mesh(radial_axis_tube, color = "red")
p.add_mesh(capitate_tube, color = "purple")
p.add_mesh(pv.Arrow(start = capitate_tube.points[0], direction = capitate_vector, scale = 10), color = "yellow")
p.add_mesh(radius_x_axis_arrow, color = "red")
p.add_mesh(radius_y_axis_arrow, color = "green")
p.add_mesh(radius_z_axis_arrow, color = "blue")
p.camera_position = "xy"

p.subplot(3, 1)
p.add_text("2D Radiocapitate Angle (Capitate Axis)\n(Simulated Sagittal X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D RC Sagittal Angle: {:.2f} °".format(radiocapitate_2d_sag_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(radius_projected_sagittal, color = "red", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(capitate_projected_sagittal, color = "purple", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(radius_2d_sag_line_projected, color = "red")
p.add_mesh(capitate_2d_sag_line_projected, color = "purple")
p.add_mesh(pv.Arrow(start = radius_2d_sag_line_projected.points[0], direction = radius_2d_sag_vector_projected, scale = 10), color = "red")
p.add_mesh(pv.Arrow(start = capitate_2d_sag_line_projected.points[0], direction = cap_2d_sag_vector, scale = 10), color = "purple")
p.camera_position, p.camera.roll, p.camera.azimuth = [camera_position_sagittal, radial_plane_origin, view_up_vector], 0, 180

p.subplot(3, 2)
p.add_text("2D Radiocapitate Angle (Capitate Axis)\n(Simulated AP X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D RC AP Angle: {:.2f} °".format(radiocapitate_2d_ap_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(radius_projected_ap, color = "red", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(capitate_projected_ap, color = "purple", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(radius_2d_ap_line_projected, color = "red")
p.add_mesh(capitate_2d_ap_line_projected, color = "purple")
p.add_mesh(pv.Arrow(start = radius_2d_ap_line_projected.points[0], direction = radius_2d_ap_vector_projected, scale = 10), color = "red")
p.add_mesh(pv.Arrow(start = capitate_2d_ap_line_projected.points[0], direction = cap_2d_ap_vector, scale = 10), color = "purple")
p.camera_position, p.camera.roll = [camera_position_ap, radial_plane_origin, view_up_vector], 0

# --------------- CAPITATE-MC3 ANGLE ---------------
p.subplot(4, 0)
p.add_text("3D Capitate-MC3 Angle", font_size = 12, position = "lower_left", color = "black")
p.add_text("3D Capitate-MC3 Angle: {:.2f} °".format(capitate_mc3_3d_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(krasl_bones["Capitate"]["Mesh"], color = "purple", opacity = 0.25)
p.add_mesh(krasl_bones["MC3"]["Mesh"], color = "orange", opacity = 0.25)
p.add_mesh(capitate_tube, color = "purple")
p.add_mesh(mc3_tube, color = "orange")
p.camera_position = "xy"

p.subplot(4, 1)
p.add_text("2D Capitate-MC3 Angle\n(Simulated Sagittal X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D Capitate-MC3 Sagittal Angle: {:.2f} °".format(capitate_mc3_2d_sag_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(capitate_projected_sagittal, color = "purple", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(mc3_projected_sagittal, color = "orange", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(capitate_2d_sag_line_projected, color = "purple")
p.add_mesh(mc3_2d_sag_line_projected, color = "orange")
p.camera_position, p.camera.roll, p.camera.azimuth = [camera_position_sagittal, radial_plane_origin, view_up_vector], 0, 180

p.subplot(4, 2)
p.add_text("2D Capitate-MC3 Angle\n(Simulated AP X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D Capitate-MC3 AP Angle: {:.2f} °".format(capitate_mc3_2d_ap_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(capitate_projected_ap, color = "purple", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(mc3_projected_ap, color = "orange", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(capitate_2d_ap_line_projected, color = "purple")
p.add_mesh(mc3_2d_ap_line_projected, color = "orange")
p.camera_position, p.camera.roll = [camera_position_ap, radial_plane_origin, view_up_vector], 0


# --------------- CAPITOLUNATE ANGLE ---------------
p.subplot(5, 0)
p.add_text("3D Capitolunate Angle", font_size = 12, position = "lower_left", color = "black")
p.add_text("3D Capitolunate Angle: {:.2f} °".format(capitolunate_3d_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(krasl_bones["Capitate"]["Mesh"], color = "purple", opacity = 0.25)
p.add_mesh(krasl_bones["Lunate"]["Mesh"], color = "yellow", opacity = 0.25)
p.add_mesh(capitate_tube, color = "purple")
p.add_mesh(lunate_tube, color = "red")
p.camera_position = "xy"

p.subplot(5, 1)
p.add_text("2D Capitolunate Angle\n(Simulated Sagittal X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D Capitolunate Sagittal Angle: {:.2f} °".format(capitolunate_2d_sag_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(capitate_projected_sagittal, color = "purple", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(lunate_projected_sagittal, color = "yellow", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(capitate_2d_sag_line_projected, color = "purple")
p.add_mesh(lunate_2d_sag_line_projected, color = "red")
p.camera_position, p.camera.roll, p.camera.azimuth = [camera_position_sagittal, radial_plane_origin, view_up_vector], 0, 180

p.subplot(5, 2)
p.add_text("2D Capitolunate Angle\n(Simulated AP X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D Capitolunate AP Angle: {:.2f} °".format(capitolunate_2d_ap_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(capitate_projected_ap, color = "purple", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(lunate_projected_ap, color = "yellow", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(capitate_2d_ap_line_projected, color = "purple")
p.add_mesh(lunate_2d_ap_line_projected, color = "red")
p.camera_position, p.camera.roll = [camera_position_ap, radial_plane_origin, view_up_vector], 0


# -------------- CAPITATE-RADIUS INDEX ---------------
p.subplot(6, 0)
p.add_text("3D Capitate-Radius Index", font_size = 12, position = "lower_left", color = "black")
p.add_text("3D Capitate-Radius Index: {:.4f} cm".format(capitate_radius_distance / 10.0), font_size = 10, position = "lower_right", color = "black")
for i, bone in enumerate(krasl_bones):
    p.add_mesh(krasl_bones[bone]["Mesh"], color="white", show_scalar_bar = False, opacity = 0.25)
p.add_mesh(krasl_bones["Radius"]["Mesh"], color = "red", opacity = 1)
p.add_mesh(krasl_bones["Capitate"]["Mesh"], color = "purple", opacity = 1)
p.add_mesh(cri_meshes_list[1], color = "white", opacity = 0.5) # The bone where projection begins from
p.add_mesh(cri_meshes_list[2], color = "black") # The minimum distance line
camera_position = "xy"
p.camera_position = camera_position

p.subplot(6, 1)
p.add_text("Capitate-Radius Index\n(Simulated AP X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("2D Capitate-Radius Index: {:.4f} cm".format(capitate_radius_line.length / 10.0), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(capitate_projected_ap, color = "purple", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(radius_projected_ap, color = "red", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(scaphoid_projected_ap, color = "green", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(lunate_projected_ap, color = "yellow", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(capitate_radius_line_projected_ap, color = "red")
p.camera_position, p.camera.roll = [camera_position_ap, radial_plane_origin, view_up_vector], 0


# --------------- SCAPHOLUNATE INTERVAL ---------------
p.subplot(7, 0)
p.add_text(f"3D Scapholunate Interval", font_size=20, position="lower_left", color="black")
p.add_mesh(scaphoid_data["Mesh"], color="white", opacity = 0.75)
p.add_mesh(lunate_data["Mesh"], color="white", opacity = 0.9)
p.add_mesh(lunate_surface_mesh, color="blue", opacity = 0.9)
colours = ["red", "orange", "yellow", "green", "blue"]
for i in range(0, 5):
    p.add_mesh(lunate_min_distance_lines[i], color = colours[i])
p.add_mesh(scapholunate_line, color = "black")
p.camera_position = "xy"
p.camera.azimuth = 30

p.subplot(7, 1)
p.add_text("2D Scapholunate Interval\n(Simulated AP X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_mesh(scaphoid_projected_ap, color = "green", opacity = 0.1)
p.add_mesh(lunate_projected_ap, color = "yellow", opacity = 0.1)
p.add_mesh(proximal_sl_line_2D, color = "red", opacity = 0.2)
p.add_mesh(palmar_sl_line_2D, color = "orange", opacity = 0.2)
p.add_mesh(distal_sl_line_2D, color = "yellow", opacity = 0.2)
p.add_mesh(dorsal_sl_line_2D, color = "green", opacity = 0.2)
p.add_mesh(central_sl_line_2D, color = "blue")
p.add_mesh(min_scapholunate_interval_line_projected_xy, color = "purple", opacity = 0.2)
p.camera_position = [camera_position_ap, radial_plane_origin, view_up_vector]
rotation = 0
p.camera.roll = rotation
p.camera.zoom(2.25)

p.subplot(7, 2)
p.add_text("Box Left Intentionally Blank", font_size = 12, position = "center", color = "black")

# --------------- DORSAL SCAPHOID TRANSLATION - DTL METHOD ---------------
p.subplot(8, 0)
p.add_text("3D Dorsal Scaphoid Translation (DTL)", font_size = 12, position = "lower_left", color = "black")
p.add_mesh(radius_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 1)
p.add_mesh(ulna_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(scaphoid_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.8)
p.add_mesh(lunate_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(capitate_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(mc3_data["Mesh"], color = "white", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(radius_x_axis_arrow, color = "red")
p.add_mesh(radius_y_axis_arrow, color = "green")
p.add_mesh(radius_z_axis_arrow, color = "blue")
p.add_mesh(scaphoid_dorsal_point, color = "red", point_size = 3)
p.add_mesh(scaphoid_dorsal_tube, color = "red", point_size = 10)
p.add_mesh(dorsal_radius_point, color = "blue", point_size = 10)
p.add_mesh(radius_rim_tube, color = "black")
camera_position = "zy"
p.camera_position = camera_position

p.subplot(8, 1)
p.add_text("2D Dorsal Scaphoid Translation - DST\n(Simulated Sagittal X-ray View)", font_size = 12, position = "lower_left", color = "black")
p.add_text("3D Dorsal Scaphoid Translation: {:.4f} mm".format(dorsal_scaphoid_translation), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(radius_projected_sagittal, color = "blue", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(scaphoid_projected_sagittal, color = "orange", show_scalar_bar = False, opacity = 0.1)
p.add_mesh(scaphoid_dorsal_line_projected.tube(radius = 0.1), color = "red")
p.add_mesh(radius_rim_line_projected.tube(radius = 0.1), color = "black")
p.add_mesh(z_axis_projected, color = "blue")
p.add_mesh(intersection_point, color = "orange")
p.add_mesh(radius_2d_sag_line_projected, color = "black")
camera_position = "zy"
p.camera_position = camera_position

# --------------- DORSAL SCAPHOID TRANSLATION - PRSA METHOD ---------------
p.subplot(8, 2)
p.add_text("PRSA 2D DST Angle", font_size = 12, position = "lower_left", color = "black")
p.add_text("PRSA-DST 2D Angle: {:.2f} °".format(prsa_2D_angle), font_size = 10, position = "lower_right", color = "black")
p.add_mesh(radius_slice, color = "red")
p.add_mesh(dorsal_line_projected, color = "blue")
p.add_mesh(radius_line_projected, color = "green")
p.camera_position, p.camera.roll, p.camera.azimuth = [camera_position_sagittal, radial_plane_origin, view_up_vector], 0, 180
p.camera.zoom(2)
p.show()

---
<a name="References"></a>
## References

The following dorsal scaphoid translation radiographic techniques were utilized in the 3D adaptations presented in the notebook and functions described above:
- *Dorsal Tangential Line (DTL) Method*: Chan K, Vutescu ES, Wolfe SW, Lee SK. 2019. Radiographs Detect Dorsal Scaphoid Translation in Scapholunate Dissociation. J Wrist Surg 08(03):186–191.
- *Posterior Radioscaphoid Angle (PRSA) Method*: Teixeira PAG, De Verbizier J, Aptel S, et al. 2016. Posterior radioscaphoid angle as a predictor of wrist degenerative joint disease in patients with scapholunate ligament tears. American Journal of Roentgenology 206(1):144–150.

<br>

The definitions of positional scapholunate intervals were based on the methodology described in the following publication:<br>
- Teule EHS, Hummelink S, Kumaş A, et al. 2024. Automatic analysis of the scapholunate distance using 4DCT imaging: normal values in the healthy wrist. Clin Radiol 79(8):e1040–e1048.

<br>

The curvature calculations in this notebook utilized the following papers/documentation as references:
- *"Estimating Curvatures and Their Derivatives on Triangle Meshes" by Szymon Rusinkiewicz (2004)* (https://gfx.cs.princeton.edu/pubs/Rusinkiewicz_2004_ECA/curvpaper.pdf)
- *"Surface Reconstruction from Unorganized Points" by Hoppe et al. (1992)* (https://graphics.pixar.com/library/Reconstruction/paper.pdf)
- *vtkCurvatures Class Reference* (https://vtk.org/doc/nightly/html/classvtkCurvatures.html#details)

<br><br>

Please see the manuscript associated with this notebook for a complete list of works referenced during this study: (Link to accepted manuscript will go here)

---
<a name="Acknowledgements"></a>
## Acknowledgements

The authors gratefully acknowledge the funding provided from the Department of Biomedical Engineering and the McCaig Institute of Bone and Joint Health at the University of Calgary, and the funding provided by the American Foundation for Surgery of the Hand through the Fast Track Basic Science Grant, the Natural Sciences and Engineering Research Council (NSERC) of Canada Discovery Grant. 


---
<a name="attribution"></a>

Notebook created using the [template](https://github.com/ORMIRcommunity/templates/blob/main/ORMIR_nb_template.ipynb) of the [ORMIR community](https://ormircommunity.github.io/) (version 1.0, 2023)